# Scenario Analysis

This notebook demonstrates how to model **multi-period** and **multi-scenario** optimizations in **flixopt**:

- **Periods** - Multiple planning years (e.g., 2020, 2021, 2022)
- **Scenarios** - Uncertain futures with different probabilities
- **Scenario weights** - Probability-weighted objective function

This is essential for:
- Long-term investment planning under uncertainty
- Sensitivity analysis
- Stochastic optimization

## Setup

In [ ]:
import numpy as np
import pandas as pd

import flixopt as fx

fx.CONFIG.notebook()

## Define Dimensions

We create a model with:
- **Timesteps**: One week at hourly resolution
- **Scenarios**: "Base Case" and "High Demand"
- **Periods**: Years 2020, 2021, 2022

In [ ]:
# Time horizon: one week
timesteps = pd.date_range('2020-01-01', periods=24 * 7, freq='h')

# Two scenarios
scenarios = pd.Index(['Base Case', 'High Demand'])

# Three planning periods (years)
periods = pd.Index([2020, 2021, 2022])

print(f'Timesteps: {len(timesteps)} hours')
print(f'Scenarios: {list(scenarios)}')
print(f'Periods: {list(periods)}')

## Create Time Series Data

### Heat Demand Profiles

Different demand patterns for each scenario:

In [ ]:
np.random.seed(42)
n_hours = len(timesteps)

# Daily patterns (24 hours) - realistic morning/evening peaks
base_daily_pattern = np.array(
    [22, 20, 18, 18, 20, 25, 40, 70, 95, 110, 85, 65, 60, 58, 62, 68, 75, 88, 105, 125, 130, 122, 95, 35]
)

high_daily_pattern = np.array(
    [28, 25, 22, 22, 24, 30, 52, 88, 118, 135, 105, 80, 75, 72, 75, 82, 92, 108, 128, 148, 155, 145, 115, 48]
)

# Tile and add random variation
base_demand = np.tile(base_daily_pattern, n_hours // 24 + 1)[:n_hours] * (1 + np.random.uniform(-0.05, 0.05, n_hours))
high_demand = np.tile(high_daily_pattern, n_hours // 24 + 1)[:n_hours] * (1 + np.random.uniform(-0.07, 0.07, n_hours))

# Create DataFrame with scenario columns
heat_demand_per_h = pd.DataFrame({'Base Case': base_demand, 'High Demand': high_demand}, index=timesteps)

heat_demand_per_h.head()

In [ ]:
# Visualize demand profiles
heat_demand_per_h.plot(figsize=(12, 4), title='Heat Demand by Scenario')

### Electricity Prices

Prices vary by period (reflecting market changes over years):

In [ ]:
# Hourly price factors (low at night, high in morning/evening)
hourly_price_factors = np.array(
    [
        0.70,
        0.65,
        0.62,
        0.60,
        0.62,
        0.70,
        0.95,
        1.15,
        1.30,
        1.25,
        1.10,
        1.00,
        0.95,
        0.90,
        0.88,
        0.92,
        1.00,
        1.10,
        1.25,
        1.40,
        1.35,
        1.20,
        0.95,
        0.80,
    ]
)

# Base prices per period (€/kWh) - reflecting energy crisis
period_base_prices = np.array([0.075, 0.095, 0.135])  # 2020, 2021, 2022

price_series = np.zeros((n_hours, 3))
for period_idx, base_price in enumerate(period_base_prices):
    price_series[:, period_idx] = (
        np.tile(hourly_price_factors, n_hours // 24 + 1)[:n_hours]
        * base_price
        * (1 + np.random.uniform(-0.03, 0.03, n_hours))
    )

power_prices = price_series.mean(axis=0)
print(f'Average electricity prices by period: {power_prices}')

### Scenario Weights

Define probability weights for each scenario. The optimizer minimizes the **expected cost**:

In [ ]:
# Scenario probabilities
scenario_weights = np.array([0.6, 0.4])  # Base: 60%, High Demand: 40%

print(f'Scenario weights: Base Case={scenario_weights[0]:.0%}, High Demand={scenario_weights[1]:.0%}')

## Create FlowSystem

The FlowSystem now includes all dimensions:

In [ ]:
flow_system = fx.FlowSystem(
    timesteps=timesteps, periods=periods, scenarios=scenarios, scenario_weights=scenario_weights
)

print(flow_system)

## Define Buses and Effects

In [ ]:
# Energy buses with automatic color assignment
flow_system.add_elements(
    fx.Bus(label='Strom', carrier='electricity'),
    fx.Bus(label='Fernwärme', carrier='heat'),
    fx.Bus(label='Gas', carrier='gas'),
)

# Cost effect with CO2 linkage
costs = fx.Effect(
    label='costs',
    unit='€',
    description='Kosten',
    is_standard=True,
    is_objective=True,
    share_from_temporal={'CO2': 0.2},  # Carbon price: 0.2 €/kg CO2
)

# CO2 emissions with hourly limit
CO2 = fx.Effect(
    label='CO2',
    unit='kg',
    description='CO2_e-Emissionen',
    maximum_per_hour=1000,
)

## Define Components

### Gas Boiler

In [ ]:
boiler = fx.linear_converters.Boiler(
    label='Boiler',
    thermal_efficiency=0.92,  # Modern condensing boiler
    thermal_flow=fx.Flow(
        label='Q_th',
        bus='Fernwärme',
        size=100,
        relative_minimum=0.1,
        relative_maximum=1,
        status_parameters=fx.StatusParameters(),
    ),
    fuel_flow=fx.Flow(label='Q_fu', bus='Gas'),
)

### CHP Unit

In [ ]:
chp = fx.linear_converters.CHP(
    label='CHP',
    thermal_efficiency=0.48,
    electrical_efficiency=0.40,
    electrical_flow=fx.Flow(
        'P_el', bus='Strom', size=80, relative_minimum=5 / 80, status_parameters=fx.StatusParameters()
    ),
    thermal_flow=fx.Flow('Q_th', bus='Fernwärme'),
    fuel_flow=fx.Flow('Q_fu', bus='Gas'),
)

### Thermal Storage

Note: Some parameters can vary by period/scenario using arrays:

In [ ]:
storage = fx.Storage(
    label='Storage',
    charging=fx.Flow('Q_th_load', bus='Fernwärme', size=1000),
    discharging=fx.Flow('Q_th_unload', bus='Fernwärme', size=1000),
    capacity_in_flow_hours=fx.InvestParameters(effects_of_investment=20, fixed_size=30, mandatory=True),
    initial_charge_state=0,
    relative_maximum_final_charge_state=np.array([0.8, 0.5, 0.1]),  # Varies by period!
    eta_charge=0.95,
    eta_discharge=0.98,
    relative_loss_per_hour=np.array([0.008, 0.015]),  # Varies by scenario!
    prevent_simultaneous_charge_and_discharge=True,
)

### Sinks and Sources

In [ ]:
# Heat demand - uses the scenario-indexed DataFrame
heat_sink = fx.Sink(
    label='Heat Demand',
    inputs=[
        fx.Flow(
            label='Q_th_Last',
            bus='Fernwärme',
            size=1,
            fixed_relative_profile=heat_demand_per_h,  # DataFrame with scenario columns
        )
    ],
)

# Gas prices vary by period (energy crisis effect)
gas_prices_per_period = np.array([0.04, 0.06, 0.11])  # €/kWh
gas_co2_emissions = 0.202  # kg CO2/kWh

gas_source = fx.Source(
    label='Gastarif',
    outputs=[
        fx.Flow(
            label='Q_Gas',
            bus='Gas',
            size=1000,
            effects_per_flow_hour={
                costs.label: gas_prices_per_period,  # Array for periods
                CO2.label: gas_co2_emissions,
            },
        )
    ],
)

# Electricity feed-in
power_sink = fx.Sink(
    label='Einspeisung',
    inputs=[
        fx.Flow(
            label='P_el',
            bus='Strom',
            effects_per_flow_hour=-1 * power_prices,  # Negative = revenue
        )
    ],
)

## Build and Visualize

In [ ]:
flow_system.add_elements(costs, CO2, boiler, storage, chp, heat_sink, gas_source, power_sink)

flow_system.topology.plot('topology.html', show=False)

## Run Optimization

In [ ]:
flow_system.optimize(fx.solvers.HighsSolver(mip_gap=0, time_limit_seconds=30))

## Analyze Results

Results now have multiple dimensions (time, scenario, period):

In [ ]:
flow_system.statistics.plot.heatmap('CHP(Q_th)')

In [ ]:
flow_system.statistics.plot.balance('Fernwärme')

In [ ]:
flow_system.statistics.plot.balance('Storage')

In [ ]:
flow_system.statistics.plot.heatmap('Storage')

### Access Multi-dimensional Data

In [ ]:
# Flow rates across all dimensions
print('Flow rates shape:', flow_system.statistics.flow_rates.dims)
flow_system.statistics.flow_rates

In [ ]:
# Charge states
flow_system.statistics.charge_states

## Summary

This example demonstrated:

- **Multi-period modeling** - Different years with evolving prices
- **Multi-scenario analysis** - Uncertain demand futures
- **Scenario weighting** - Expected value optimization
- **Dimension-varying parameters** - Arrays for period/scenario-specific values

Key concepts:
- `periods` index represents planning years
- `scenarios` index represents uncertainty
- `scenario_weights` define probabilities
- Parameters can be arrays matching dimension sizes
- DataFrames can have scenario-indexed columns